In [20]:
import pandas as pd

# Load the anonymized DataFrame
df = pd.read_pickle('data/output/anonymized_data.pkl')

idx = 8816

df

{8816: {'metadata': {'topic_title': 'Audition CS3? Nogle erfaringer?',
   'forum_name': 'Computer & Software',
   'total_posts': 3},
  'posts': [{'post_id': 88623.0,
    'poster_id': 1135.0,
    'post_number': 0,
    'post_time': 1199360677.0,
    'original_text': '<t>Hey!<br/>\n<br/>\nSå at der var kommet et nyt Adobe Audition og at den nu understøttede VSTi ? Kan det passe?<br/>\nFik indtrykket af at det nu fungerer på samme måde som Cubase og Logic?<br/>\n<br/>\nNogle der har prøvet det, bruger det eller?<br/>\n<br/>\nSer da spændende ud!<br/>\n<br/>\nKAYN!</t>',
    'normalized_text': 'Hey!  Så at der var kommet et nyt Adobe Audition og at den nu understøttede VSTi ? Kan det passe? Fik indtrykket af at det nu fungerer på samme måde som Cubase og Logic?  Nogle der har prøvet det, bruger det eller?  Ser da spændende ud!  KAYN!',
    'ner_anonymized_text': 'Hey!  Så at der var kommet et nyt Adobe Audition og at den nu understøttede VSTi ? Kan det passe? Fik indtrykket af at det nu fun

In [21]:
df[idx]['posts'][2]['anonymized_text']

'Har bare brugt de tidligere versioner til noget wav editering, bending osv.. Så bare nysgerrig om nogle brugte den nye version :)'

In [23]:
print(df[idx]['posts'][0]['anonymized_text'])

Hey!  Så at der var kommet et nyt Adobe Audition og at den nu understøttede VSTi ? Kan det passe? Fik indtrykket af at det nu fungerer på samme måde som Cubase og Logic?  Nogle der har prøvet det, bruger det eller?  Ser da spændende ud!  [PERSON]!


In [24]:
import pandas as pd

# Load the anonymized DataFrame
df = pd.read_pickle('data/output/anonymized_data_df.pkl')
df


,conversation_id,post_number,post_id,poster_id,post_time,forum_name,topic_title,total_posts_in_conversation,original_text,normalized_text,anonymized_text,original_text_tokens,normalized_text_tokens,anonymized_text_tokens
77,4436,0,45158,343,1.177102e+09,Branche & Teori,Marilyn Manson,6,"<r>Prøv en gang at ta et lyt til fx. ""The Spee...","Prøv en gang at ta et lyt til fx. ""The Speed o...","Prøv en gang at ta et lyt til fx. ""The Speed o...",125,108,108
78,4436,0,45211,2,1.177146e+09,Branche & Teori,Marilyn Manson,6,<t>Det er et konge album. Har ikke lige checke...,Det er et konge album. Har ikke lige checket d...,Det er et konge album. Har ikke lige checket d...,38,33,33
79,4436,0,45228,396,1.177149e+09,Branche & Teori,Marilyn Manson,6,"<r><QUOTE author=""Simond""><s>[quote=""Simond""]<...",Pænt sej sangteknik.,Pænt sej sangteknik.,84,8,8
80,4436,0,45251,2,1.177151e+09,Branche & Teori,Marilyn Manson,6,<r><E>:lol:</E></r>,:lol:,:lol:,12,4,4
81,4436,0,45272,343,1.177155e+09,Branche & Teori,Marilyn Manson,6,<r>Pent sej sangteknik <E>:lol:</E> Det har h...,Pent sej sangteknik :lol: Det har han helt sik...,Pent sej sangteknik :lol: Det har han helt sik...,81,71,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,51633,0,483250,22060,1.360015e+09,Computer & Software,Søger hjælp til ny computer.,25,<r>SLL. ! Du er bare great- og tak fordi du gi...,SLL. ! Du er bare great- og tak fordi du gider...,SLL[PERSON] ! Du er bare great- og tak fordi d...,127,99,101
73,51633,0,483252,193,1.360015e+09,Computer & Software,Søger hjælp til ny computer.,25,"<r><QUOTE author=""Heine""><s>[quote=""Heine""]</s...",Jeg tog blot udgangspunkt i hvad jeg kunne se ...,Jeg tog blot udgangspunkt i hvad jeg kunne se ...,2227,935,901
74,51633,0,483253,15833,1.360015e+09,Computer & Software,Søger hjælp til ny computer.,25,"<r><QUOTE author=""Heine""><s>[quote=""Heine""]</s...","Ja, -V LX har vist bevist den ligger i bedste ...","Ja, -V LX har vist bevist den ligger i bedste ...",172,99,99
75,51633,0,483254,193,1.360015e+09,Computer & Software,Søger hjælp til ny computer.,25,"<r><QUOTE author=""Heine""><s>[quote=""Heine""]</s...","Ja, -V LX har vist bevist den ligger i bedste ...","Ja, -V LX har vist bevist den ligger i bedste ...",240,138,138


In [25]:
import pandas as pd

# Load the anonymized DataFrame
df = pd.read_pickle('data/processed_data_500.pkl')
df

,topic_id,topic_title,topic_poster,forum_id,forum_name,post_id,post_text,post_time,poster_id,post_subject,post_number,conversation_id
1129,386,Extern harddisk til musikbrug,148,1,Computer & Software,2987.0,"<t>Da jeg snart køber mig en MBP, så skal jeg ...",1.147810e+09,148.0,Extern harddisk til musikbrug,0,386
1130,386,Extern harddisk til musikbrug,148,1,Computer & Software,2990.0,<r>Har en Seagate Barracuda (200 GB) i et Fire...,1.147811e+09,21.0,,1,386
1131,386,Extern harddisk til musikbrug,148,1,Computer & Software,2993.0,"<r>Jeg kører en Maxtor på 250 Gb. på 3'die år,...",1.147811e+09,75.0,,2,386
1132,386,Extern harddisk til musikbrug,148,1,Computer & Software,2999.0,<t>Lacie D2 Extreme ... hurtige diske og kanon...,1.147813e+09,91.0,,3,386
1133,386,Extern harddisk til musikbrug,148,1,Computer & Software,3000.0,"<t>har købt en 7200 spin, HD der har ""bærbar"" ...",1.147813e+09,106.0,,4,386
...,...,...,...,...,...,...,...,...,...,...,...,...
1254,85371,Rme fireface og wa 2a problemer,32084,2,Andet Udstyr & Lydteknik,745191.0,<r>Start med at tage compen helt væk fra ligni...,1.693803e+09,1372.0,Re: Rme fireface og wa 2a problemer,1,85371
1255,85371,Rme fireface og wa 2a problemer,32084,2,Andet Udstyr & Lydteknik,745231.0,<t>Den har jeg også prøvet uden held desværre....,1.693892e+09,32084.0,Re: Rme fireface og wa 2a problemer,2,85371
1256,85371,Rme fireface og wa 2a problemer,32084,2,Andet Udstyr & Lydteknik,745233.0,<r>Og hvad er resultatet der? Du får ingen lyd...,1.693896e+09,1372.0,Re: Rme fireface og wa 2a problemer,3,85371
760,85765,Cobra ACA016 hum-isolator ???,21671,2,Andet Udstyr & Lydteknik,748574.0,"<r>100 kr<br/>\n<br/>\n<URL url=""https://www.s...",1.705167e+09,21671.0,Cobra ACA016 hum-isolator ???,0,85765
